### Used the Galaxy manipulation tool `Filter sequences by ID from a tabular file`to get just the _T.wesiss_ protein sequences that are 1) found on Day 0 and 2) from the cytoplasm.

### PROTEUS2 protein secondary structure analysis of remanent (survivor) proteins:

### This is a notebook to combine and evaluate the protein secondary structure output for Day 0 cytoplasm protein sequences.

### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/database-fastas/

/home/millieginty/Documents/git-repos/rot-mayer/data/database-fastas


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

### Galaxy wasn't letting me download the filtered fasta (see above), so I did so manually for the Day 0 DB diatom cytoplasm proteins. Called `T0-322-trypsin-DB-diatom-cytoplasm.fasta`.

### Need to change all the `I` residues to `L` in the original fasta so I can manually search for protein IDs using the UniPept LCA peptides (which don't have `I`'s). 

### Still have to remove any `X` residues, though, because Proteus2 will not accept sequences with them. Using `sed`:

In [3]:
ls

mmetsp_uniprot_tw_combo.fasta  mmetsp_uniprot_tw_combo_singleline.fasta


In [5]:
# use sed to find all I's and replace with L's

!sed 's/I/L/g' mmetsp_uniprot_tw_combo_singleline.fasta > mmetsp_uniprot_tw_combo_singleline_IL.fasta

In [6]:
# use sed to remove all occurances of X

!sed 's/X//g' mmetsp_uniprot_tw_combo_singleline_IL.fasta > mmetsp_uniprot_tw_combo_singleline_IL_noX.fasta

In [6]:
# take a look

!head -30 T12-325-trypsin-DB-diatom-membrane-noX.fasta

>Thalassiosira_weissflogii_0171421366 
MKSAIIASLIAGAAAFAPTQTGRTPTSVSESKADLEAIASKANPFVKFYDPLNLAEQEFWGKSNEETIAWLRQSEIKHGRIAMFAFVGYIVQSNFVFPWAQTLAGAPHPSADLSPEAQWDAIPLGAKWQIFAVISMLELWDECGGGGAMPHYTKGRQPGKYPPFTLFRENVHFVLDLYDPFGFNKNMSAETKERRLVAELNNGRLAQLGIFGFLCADKIPGSVPVLNSIAIPYDGNPMIPFEGQFSYFN
>Thalassiosira_weissflogii_0201829098 
MMKLAALAALAGSAAAFAPSTGGRAVTSLSEKSVSLPFLETPPNTAGYVGDVGFDPFRFSDFIPVDFLREAELKHGRICMMAWLGFVAVDCGFRVYPIPEGFEGITAVQAHDAAVEQGGLSQMLLWFSLAEVFDLIAVVQMLEGSGRAPGDFGLDGGFLKGKSAAEIETMKLKEITHCRLAMFAFSGVVTQAVLTQGPFPYV
>Thalassiosira_weissflogii_0201860448 
HQRNLKMKLAVLAALFGSAAAFAPAQTGKATSALNAFESELGAQPPLGFFDPLGLLDDADQERFDRLRYVEIKHGRIAQLAFLGNIITRAGVHLPGNIDYAGNSFDSFPNGWAAILDAAFAPAQTGKATSALNAFESELGAQPPLGFFDPLGLLDDADQERFDRLRYVEIKHGRIAQLAFLGNIITRAGVHLPGNIDYAGNSFDSFPNGWAAISGPDAISGSGLGQIVAFVGFLELFVMKDVTGEGEFVGDFRNGALDFGWDKFDAETKLSKRAIELNNGRAAMMGILGLMVHEQLGGSLPIVGEM
>Thalassiosira_weissflogii_0203152572 
WAAISGPDAISGSGLGQIVAFVGFLELFVMKDVTGEGEFVGDFRNGALDFGWDKFDAETKLSKRAIELNNGRAAMMGILGLMVH

### This worked! Proteus2 accepted the submission. Now the results come in an email from Proteus2 in a garbage format:

### The output of Proteus2 comes in an email in nominally FASTA format, but with extra line breaks and spacings within protein and prediction sequences. Also, the name gets cut off.

#### They look like this:

#### >Thalas

MMKLAALAAL MGSAAAFAPA QTGKASTQLR AFEDELGAQP PLGFFDPFGM 

CCHHHHHHHH HHHHHCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

LSGDCTQERF DRLRYVEIKH GRICMLAFLG QIVTRAGIHL PGSINYAGDS 

CCCCCCHHHH HHHHHHHHHH HHHHHHHHHH HHHHHHHCCC CCCCCCCCCC 

FDSFPNGVAA LFGPNSIPTA GLVQIIAFIG VLECAFMRDV PGTGNEFVGD 

CCCCCCCCCC CCCCCCCCHH HHHHHHHHHH HHHHHHHHCC CCCCCCCCCC 

FRNGYIDFGW DDFDEETKLQ KRAIQSGTIS NMMKLAALAA LMGSAAAFAP 

CCCCCCCCCC CCCCHHHHHH HHHHHHHHHH HHHHHHHHHC CCCCCCCCCC 



### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [13]:
!head T12-325-trypsin-DB-diatom-membrane-proteins

>Thalas

MKSAIIASLI AGAAAFAPTQ TGRTPTSVSE SKADLEAIAS KANPFVKFYD 
CCHHHHHHHH HHCCCCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

PLNLAEQEFW GKSNEETIAW LRQSEIKHGR IAMFAFVGYI VQSNFVFPWA 
CCCCCCCCCC CCCCHHHHHH HHHHHHHHHH HHHHHHHHHH HHHHHCCCCC 

QTLAGAPHPS ADLSPEAQWD AIPLGAKWQI FAVISMLELW DECGGGGAMP 
CCCCCCCCCC CCCCCCCCCC CCCHHHHHHH HHHHHHHHHH HHHHHHHHCC 


In [14]:
# remove empty lines
# remove empty spaces from lines
# remove carat protein sequence names
# collapse the 2 lines from every protein sequence line (protein seq, secondard stucture pred.) into one line

!sed '/^[[:space:]]*$/d' T12-325-trypsin-DB-diatom-membrane-proteins | cat \
| sed '/>/d' \
| tr -d "[:blank:]" > T12-325-trypsin-DB-diatom-membrane-proteins.txt

!awk '{printf "%s%s",$0,(NR%2?FS:RS)}' T12-325-trypsin-DB-diatom-membrane-proteins.txt > \
T12-325-trypsin-DB-diatom-membrane-proteins_sort.csv

In [15]:
!head T12-325-trypsin-DB-diatom-membrane-proteins_sort.csv

MKSAIIASLIAGAAAFAPTQTGRTPTSVSESKADLEAIASKANPFVKFYD CCHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
PLNLAEQEFWGKSNEETIAWLRQSEIKHGRIAMFAFVGYIVQSNFVFPWA CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCC
QTLAGAPHPSADLSPEAQWDAIPLGAKWQIFAVISMLELWDECGGGGAMP CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCC
HYTKGRQPGKYPPFTLFRENVHFVLDLYDPFGFNKNMSAETKERRLVAEL CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHH
NNGRLAQLGIFGFLCADKIPGSVPVLNSIAIPYDGNPMIPFEGQFSYFN HHHHHHHHHHHHHHHHHHHCCCCCEECCCCCCCCCCCCEEEECCCCCCC
MMKLAALAALAGSAAAFAPSTGGRAVTSLSEKSVSLPFLETPPNTAGYVG CCHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
DVGFDPFRFSDFIPVDFLREAELKHGRICMMAWLGFVAVDCGFRVYPIPE CCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCC
GFEGITAVQAHDAAVEQGGLSQMLLWFSLAEVFDLIAVVQMLEGSGRAPG CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCC
DFGLDGGFLKGKSAAEIETMKLKEITHCRLAMFAFSGVVTQAVLTQGPFP CCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCC
HQRNLKMKLAVLAALFGSAAAFAPAQTGKATSALNAFESELGAQPPLGFF CCEEEEEHHHHHHHHHHHHHCCCCCCCCCCCCC

In [17]:
# read into pandas with space delimeter

PeaksDB_325_mem_prot = pd.read_csv("T12-325-trypsin-DB-diatom-membrane-proteins_sort.csv", delim_whitespace=True, header=None)


# name columns

PeaksDB_325_mem_prot.columns =['Stripped protein sequence', 'Secondary structure pred.'] 

In [19]:
PeaksDB_325_mem_prot.head(13)

,Stripped protein sequence,Secondary structure pred.
0,MKSAIIASLIAGAAAFAPTQTGRTPTSVSESKADLEAIASKANPFV...,CCHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
1,PLNLAEQEFWGKSNEETIAWLRQSEIKHGRIAMFAFVGYIVQSNFV...,CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHC...
2,QTLAGAPHPSADLSPEAQWDAIPLGAKWQIFAVISMLELWDECGGG...,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHH...
3,HYTKGRQPGKYPPFTLFRENVHFVLDLYDPFGFNKNMSAETKERRL...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHH...
4,NNGRLAQLGIFGFLCADKIPGSVPVLNSIAIPYDGNPMIPFEGQFSYFN,HHHHHHHHHHHHHHHHHHHCCCCCEECCCCCCCCCCCCEEEECCCCCCC
5,MMKLAALAALAGSAAAFAPSTGGRAVTSLSEKSVSLPFLETPPNTA...,CCHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
6,DVGFDPFRFSDFIPVDFLREAELKHGRICMMAWLGFVAVDCGFRVY...,CCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCC...
7,GFEGITAVQAHDAAVEQGGLSQMLLWFSLAEVFDLIAVVQMLEGSG...,CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCC...
8,DFGLDGGFLKGKSAAEIETMKLKEITHCRLAMFAFSGVVTQAVLTQ...,CCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCC...
9,HQRNLKMKLAVLAALFGSAAAFAPAQTGKATSALNAFESELGAQPP...,CCEEEEEHHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCC...


In [20]:
# add a column with the stripped peptide length (number of AAs)
PeaksDB_325_mem_prot['Sequence length'] = PeaksDB_325_mem_prot['Stripped protein sequence'].apply(len)

# use a count function to enumerate the # of C's (coil residues) for each peptide
PeaksDB_325_mem_prot['C'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("C")

# use a count function to enumerate the # of H's (helices residues) in each peptide
PeaksDB_325_mem_prot['H'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("H")

# use a count function to enumerate the # of E's (beta strand residues) in each peptide
PeaksDB_325_mem_prot['E'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("E")

#use a count function to enumerate the # of T's (membrane helix residues) in each peptide
PeaksDB_325_mem_prot['T'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("T")

# use a count function to enumerate the # of B's (membrane strand residues) in each peptide
PeaksDB_325_mem_prot['B'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("B")

# use a count function to enumerate the # of B's (signal peptide residues) in each peptide
PeaksDB_325_mem_prot['S'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("S")

# use a count function to enumerate the # of c's (cleavage site residues) in each peptide
PeaksDB_325_mem_prot['c'] = PeaksDB_325_mem_prot['Secondary structure pred.'].str.count("c")

# add a column with the % C
PeaksDB_325_mem_prot['% C'] = PeaksDB_325_mem_prot['C'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % H
PeaksDB_325_mem_prot['% H'] = PeaksDB_325_mem_prot['H'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % E
PeaksDB_325_mem_prot['% E'] = PeaksDB_325_mem_prot['E'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % T
PeaksDB_325_mem_prot['% T'] = PeaksDB_325_mem_prot['T'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % B
PeaksDB_325_mem_prot['% B'] = PeaksDB_325_mem_prot['B'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % S
PeaksDB_325_mem_prot['% S'] = PeaksDB_325_mem_prot['S'] / PeaksDB_325_mem_prot['Sequence length']

# add a column with the % c
PeaksDB_325_mem_prot['% c'] = PeaksDB_325_mem_prot['c'] / PeaksDB_325_mem_prot['Sequence length']

# additive check

PeaksDB_325_mem_prot['% check'] = PeaksDB_325_mem_prot['% C'] + PeaksDB_325_mem_prot['% H'] + PeaksDB_325_mem_prot['% E'] \
                                + PeaksDB_325_mem_prot['% T'] + PeaksDB_325_mem_prot['% B'] + \
                                PeaksDB_325_mem_prot['% S'] + PeaksDB_325_mem_prot['% c']

In [21]:
PeaksDB_325_mem_prot.head()

,Stripped protein sequence,Secondary structure pred.,Sequence length,C,H,E,T,B,S,c,% C,% H,% E,% T,% B,% S,% c,% check
0,MKSAIIASLIAGAAAFAPTQTGRTPTSVSESKADLEAIASKANPFV...,CCHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,50,40,10,0,0,0,0,0,0.800000,0.200000,0.000000,0.0,0.0,0.0,0.0,1.0
1,PLNLAEQEFWGKSNEETIAWLRQSEIKHGRIAMFAFVGYIVQSNFV...,CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHC...,50,19,31,0,0,0,0,0,0.380000,0.620000,0.000000,0.0,0.0,0.0,0.0,1.0
2,QTLAGAPHPSADLSPEAQWDAIPLGAKWQIFAVISMLELWDECGGG...,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHH...,50,25,25,0,0,0,0,0,0.500000,0.500000,0.000000,0.0,0.0,0.0,0.0,1.0
3,HYTKGRQPGKYPPFTLFRENVHFVLDLYDPFGFNKNMSAETKERRL...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHH...,50,38,12,0,0,0,0,0,0.760000,0.240000,0.000000,0.0,0.0,0.0,0.0,1.0
4,NNGRLAQLGIFGFLCADKIPGSVPVLNSIAIPYDGNPMIPFEGQFSYFN,HHHHHHHHHHHHHHHHHHHCCCCCEECCCCCCCCCCCCEEEECCCCCCC,49,24,19,6,0,0,0,0,0.489796,0.387755,0.122449,0.0,0.0,0.0,0.0,1.0


In [22]:
index = ['322 total']

data = {
        '% C total': PeaksDB_325_mem_prot['% C'].sum(),
        '% H total': PeaksDB_325_mem_prot['% H'].sum(),
        '% E total': PeaksDB_325_mem_prot['% E'].sum(),
        '% T total': PeaksDB_325_mem_prot['% T'].sum(),
        '% B total': PeaksDB_325_mem_prot['% B'].sum(),
        '% S total': PeaksDB_325_mem_prot['% S'].sum(),
        '% c total': PeaksDB_325_mem_prot['% c'].sum(),
        '% check sum': PeaksDB_325_mem_prot['% check'].sum()
       }

PeaksDB_325_mem_prot_totals = pd.DataFrame(data, columns=['% C total', '% H total', '% E total', '% T total', \
                                                      '% B total',  '% S total', '% c total', \
                                                      '% check sum'], index=index)

PeaksDB_325_mem_prot_totals['overall % sum'] = PeaksDB_325_mem_prot_totals['% C total'] \
                                            + PeaksDB_325_mem_prot_totals['% H total'] \
                                            + PeaksDB_325_mem_prot_totals['% E total'] \
                                            + PeaksDB_325_mem_prot_totals['% T total'] \
                                            + PeaksDB_325_mem_prot_totals['% B total'] \
                                            + PeaksDB_325_mem_prot_totals['% S total'] \
                                            + PeaksDB_325_mem_prot_totals['% c total'] 


PeaksDB_325_mem_prot_totals['overall % C'] = PeaksDB_325_mem_prot_totals['% C total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % H'] = PeaksDB_325_mem_prot_totals['% H total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % E'] = PeaksDB_325_mem_prot_totals['% E total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % T'] = PeaksDB_325_mem_prot_totals['% T total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % B'] = PeaksDB_325_mem_prot_totals['% B total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % S'] = PeaksDB_325_mem_prot_totals['% S total'] / PeaksDB_325_mem_prot_totals['overall % sum']

PeaksDB_325_mem_prot_totals['overall % c'] = PeaksDB_325_mem_prot_totals['% c total'] / PeaksDB_325_mem_prot_totals['overall % sum']

# write to csv

PeaksDB_325_mem_prot_totals.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/Day12_325 PeaksDB_dia_mem_prot_trypsin_totals")

PeaksDB_325_mem_prot_totals.head()

,% C total,% H total,% E total,% T total,% B total,% S total,% c total,% check sum,overall % sum,overall % C,overall % H,overall % E,overall % T,overall % B,overall % S,overall % c
322 total,34.002081,24.316742,2.025002,6.656174,0.0,0.0,0.0,67.0,67.0,0.507494,0.362936,0.030224,0.099346,0.0,0.0,0.0
